In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [6]:
train=datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=None,
    target_transform=None,
)
test=datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=None,
    target_transform=None,
)
